# Installation Instructions

Clone the repos, install dependencies and install packages terrapin, quest and gsshapy.

Note: terrapin isn't used in this example but it will be in future examples...

```bash
git clone git@public.git.erdc.dren.mil:computational-analysis-and-mechanics/quest.git
git clone git@public.git.erdc.dren.mil:computational-analysis-and-mechanics/terrapin.git
git clone git@github.com:CI-WATER/gsshapy.git

conda env create -n earthsim -f ./terrapin/py3_conda_environment.yml
conda env update -n earthsim -f ./quest/py3_conda_environment.yml
conda env update -n earthsim -f ./gsshapy/conda_env.yml
source activate earthsim

cd terrapin
python setup.py develop
cd ../quest
python setup.py develop
cd ../gsshapy
python setup.py develop
```


In [1]:
from datetime import datetime, timedelta
import os

import geopandas as gpd
from gsshapy.modeling import GSSHAModel
import quest

/Users/rditldp9/miniconda3/envs/earthsim/lib/python3.5/site-packages/xarray/core/formatting.py:16: FutureWarning: The pandas.tslib module is deprecated and will be removed in a future version.
  from pandas.tslib import OutOfBoundsDatetime


Function to automate the downloading process:

In [2]:
def download_data(service_uri, bounds, collection_name):
    """
    Downloads raster data from source uri and adds to a quest collection.
    
    If multiple raster tiles are retrieved for the given bounds it calls a quest 
    filter to merge the tiles into a single raster.
    """
    
    # download the features (i.e. locations) and metadata for the given web service
    # the first time this is run it will take longer since it downloads 
    # and caches all metadata for the given service.
    features = quest.api.get_features(uris=service_uri,
                                      filters={'bbox': bounds},
                                      as_dataframe=True,
                                      )
    
    # add the selected features to a quest collection
    added_features = quest.api.add_features(collection_name, features)
    
    # stage the data for download, optional parameters can be provided here 
    # for certain web services (i.e. date range etc), raster services don't 
    # typically have any optional parameters.
    staged_datasets = quest.api.stage_for_download(uris=added_features)
    
    # download the staged datasets
    quest.api.download_datasets(datasets=staged_datasets)
    final_datasets = staged_datasets
    
    # if more than one raster tile downloaded, merge into a single raster tile
    if len(staged_datasets) > 1:
        merged_datasets = quest.api.apply_filter(name='raster-merge',
                                                 datasets=staged_datasets)
        final_datasets = merged_datasets['datasets']
        # delete the original individual tiles
        quest.api.delete(staged_datasets)

    # return file path of final geotiff
    downloaded_datasets = quest.api.get_metadata(final_datasets)
    downloaded_file_paths = [dataset['file_path'] for dataset
                             in downloaded_datasets.values()]
    return downloaded_file_paths[0]

Parameters to change for the run:

In [4]:
elevation_service = 'svc://usgs-ned:13-arc-second'
land_use_service = 'svc://usgs-nlcd:2011'
land_use_grid_id = 'nlcd'
base_dir = './' # use local dir for now, assumes notebook is launched from the examples folder
boundary_shapefile = os.path.join(base_dir, 'vicksburg_watershed', 'watershed_boundary.shp')
gssha_model_name = 'vicksburg_south'
gssha_model_directory = os.path.join(base_dir, gssha_model_name)

Get the download boundary from the shapefile:

In [5]:
boundary_gpd = gpd.read_file(boundary_shapefile)
bounds = [str(x) for x in boundary_gpd.geometry.bounds.values[0]]

The collection is where we will origanize the downloaded data.

In [6]:
try:
    quest.api.new_collection(gssha_model_name)
except ValueError as e:
    print(e)

Download the data. This may take some time.

In [7]:
elevation_file_path = download_data(elevation_service, bounds, gssha_model_name)

retrieving raster availability from https://www.sciencebase.gov/catalog/items?parentId=4f70aa9fe4b058caae3f8de5&filter=tags%3DIMG&max=1000&fields=webLinks%2Cspatial%2Ctitle&format=json&filter=spatialQuery%3D%7Bwkt%3A%22POLYGON+%28%28-180.0+90.0%2C-180.0+-90.0%2C180.0+-90.0%2C180.0+90.0%2C-180.0+90.0%29%29%22%2Crelation%3A%22intersects%22%7D
retrieving raster availability from https://www.sciencebase.gov/catalog/items?filter=tags%3DIMG&filter=spatialQuery%3D%7Bwkt%3A%22POLYGON+%28%28-180.0+90.0%2C-180.0+-90.0%2C180.0+-90.0%2C180.0+90.0%2C-180.0+90.0%29%29%22%2Crelation%3A%22intersects%22%7D&max=1000&format=json&fields=webLinks%2Cspatial%2Ctitle&parentId=4f70aa9fe4b058caae3f8de5&offset=1000
... ... .img format raster saved at /Users/rditldp9/Library/Application Support/quest/projects/default/vicksburg_south/d5d5580a718d48058392136c78f4f531.img
... ... .img format raster saved at /Users/rditldp9/Library/Application Support/quest/projects/default/vicksburg_south/d7f0dd58ad5d4c14aac09e27514

In [8]:
land_use_file_path = download_data(land_use_service, bounds, gssha_model_name)

... ... .tif format raster saved at /Users/rditldp9/Library/Application Support/quest/projects/default/vicksburg_south/d2ffc05b6b3d44d2aacc6dbf351bd8a4.tif


Use the data to generate a GSSHA model. This may take a minute.

In [9]:
# make the directory for the output
try:
    os.mkdir(gssha_model_directory)
except OSError:
    pass

# generate GSSHA model files
model = GSSHAModel(project_name=gssha_model_name,
                   project_directory=gssha_model_directory,
                   mask_shapefile=boundary_shapefile,
                   elevation_grid_path=elevation_file_path,
                   land_use_grid=land_use_file_path,
                   land_use_grid_id=land_use_grid_id,
                   out_hydrograph_write_frequency=1,
                   )

# add card for max depth
model.project_manager.setCard('FLOOD_GRID',
                              '{0}.fgd'.format(gssha_model_name),
                              add_quotes=True)
# TODO: Add depth grids to simulation
# MAP_FREQ, DEPTH

# add event for simulation
model.set_event(simulation_start=datetime.utcnow(),
                simulation_duration=timedelta(seconds=2*60),
                rain_intensity=24,
                rain_duration=timedelta(seconds=1*60),
                )
model.write()

AttributeError: 'NoneType' object has no attribute 'GetLayer'